<a href="https://colab.research.google.com/github/Devdeep-J-S/Vision-Transformers-CMS/blob/main/Task_1_Electron_photon_classification_Keras_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name : Devdeep Shetranjiwala <br>
Email ID : devdeep0702@gmail.com 

## Task 1. Electron/photon classification
Datasets:</br>
https://cernbox.cern.ch/index.php/s/AtBT8y4MiQYFcgc (photons) </br>
https://cernbox.cern.ch/index.php/s/FbXw3V4XNyYB3oA (electrons) </br>
> Description: </br>
32x32 matrices (two channels - hit energy and time) for two classes of particles electrons and photons impinging on a calorimeter
Please use a deep learning method of your choice to achieve the highest possible
classification on this dataset.

>In this task, we will use deep learning to classify two classes of particles: electrons and photons impinging on a calorimeter. We will use two datasets, one for photons and one for electrons, which contains 32x32 matrices (two channels - hit energy and time) for each particle.</br>
We will usw deep learning framework to implement our solution, Keras/TensorFlow. Our goal is to achieve the highest possible classification accuracy on this dataset with a ROC AUC score of at least 0.80.
</br>
First, we will load the data and preprocess it.<br>
Data Preprocessing : </br>
We will load the datasets for photons and electrons and preprocess them. We will convert the data into numpy arrays and normalize them by dividing each pixel value by the maximum pixel value.

In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import sys

In [ ]:
print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num of GPUs Available:  1


In [ ]:
#Getting data
import requests
url = 'https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
r = requests.get(url, allow_redirects=True)
open('photons.hdf5', 'wb').write(r.content)
url = 'https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
r = requests.get(url, allow_redirects=True)
open('electrons.hdf5', 'wb').write(r.content)

128927319

>Model definition:</br>
Keras/TensorFlow</br>
We will define a simple convolutional neural network (CNN) with two convolutional layers, followed by a max pooling layer, and two fully connected layers. We will use the sigmoid activation function in the last layer as this is a binary classification problem.

In [ ]:
class Net(tf.keras.Model):
    def __init__(self, skip_connection=False):
        super().__init__()
        self.skip_connection = skip_connection
        self.rflip = tf.keras.layers.RandomFlip(mode="horizontal_and_vertical")
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')
        self.conv1p1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same')
        self.maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.batchnorm1 = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')
        self.conv2p1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same')
        self.maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.batchnorm2 = tf.keras.layers.BatchNormalization()
        
        self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')
        self.conv3p1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same')
        self.maxpool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.batchnorm3 = tf.keras.layers.BatchNormalization()
        
        self.conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=1, padding='same', activation='relu')
        
        self.gloavgpool = tf.keras.layers.GlobalAveragePooling2D()
        
        self.dense1 = tf.keras.layers.Dense(1, activation='sigmoid')
        self.dropout1 = tf.keras.layers.Dropout(.1)
    
    def build_graph(self):
        x = tf.keras.layers.Input(shape=train_x.shape[1:])
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

    def call(self, inputs):
        x = self.rflip(inputs)
        
        x_res = self.conv1(x)
        x = self.conv1p1(x_res)
        if self.skip_connection: x = x+x_res
        x = self.maxpool1(x)
        x = self.batchnorm1(tf.keras.layers.ReLU()(x))
        
        x_res = self.conv2(x)
        x = self.conv2p1(x_res)
        if self.skip_connection: x = x+x_res
        x = self.maxpool2(x)
        x = self.batchnorm2(tf.keras.layers.ReLU()(x))
        
        x_res = self.conv3(x)
        x = self.conv3p1(x_res)
        if self.skip_connection: x = x+x_res
        x = self.maxpool3(x)
        x = self.batchnorm3(tf.keras.layers.ReLU()(x))
        
        x = self.conv4(x)
        
        x = self.dropout1(self.gloavgpool(x))
        
        return self.dense1(x)


In [ ]:
#read the hdf5 fies
e_set = h5py.File('./electrons.hdf5', 'r')
p_set = h5py.File('./photons.hdf5', 'r')

#convert to np arrays
e_x, p_x = np.asarray(e_set['X']), np.asarray(p_set['X'])

del(e_set,p_set)
#concat the electon/photon arrays
ep_x = np.concatenate([e_x, p_x])
ep_target = np.concatenate([np.ones(len(e_x)), np.zeros(len(p_x))])

del(e_x,p_x)
#remove entries with all zeros
nonzeros = np.sum(ep_x, axis=(1,2,3))!=0
ep_x, ep_target = ep_x[nonzeros], ep_target[nonzeros]

#set seed for reproducibility
seed = 42

#split into train (80%) /test (20%) set and save
X_train, X_test, y_train, y_test = train_test_split(ep_x, ep_target, test_size=0.2, stratify=ep_target, random_state=seed)

del(ep_x,ep_target)
#normalize data with training set mean and std to ensure no data leakage
X_train_mean, X_train_std = X_train.mean((0,1,2)), X_train.std((0,1,2))
X_train = (X_train-X_train_mean)/X_train_std
X_test = (X_test-X_train_mean)/X_train_std
del(X_train_mean,X_train_std)

#split into train (60% train , 20% validate , 20% test) -> 80 / 20 
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train, test_size=0.2, stratify=y_train, random_state=seed)

train_x, train_y = X_train, y_train
del(X_train,y_train)
valid_x, valid_y = X_valid, y_valid
del(X_valid,y_valid)
test_x, test_y = X_test,y_test
del(X_test,y_test)

In [ ]:
model = Net()
model.build_graph().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 2)]       0         
                                                                 
 random_flip (RandomFlip)    (None, 32, 32, 2)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1216      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 re_lu (ReLU)                (None, 16, 16, 64)        0         
                                                             

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                    tf.keras.metrics.AUC(name='auc')])

In [ ]:
history = model.fit(x=train_x, y=train_y, batch_size=32, epochs=100, validation_data=(valid_x, valid_y))

Epoch 1/100
9949/9949 [==============================] - 106s 9ms/step - loss: 0.6017 - binary_accuracy: 0.6800 - auc: 0.7363 - val_loss: 0.5695 - val_binary_accuracy: 0.7106 - val_auc: 0.7739
Epoch 2/100
9949/9949 [==============================] - 89s 9ms/step - loss: 0.5699 - binary_accuracy: 0.7112 - auc: 0.7739 - val_loss: 0.5699 - val_binary_accuracy: 0.7108 - val_auc: 0.7818
Epoch 3/100
9949/9949 [==============================] - 89s 9ms/step - loss: 0.5627 - binary_accuracy: 0.7178 - auc: 0.7813 - val_loss: 0.5579 - val_binary_accuracy: 0.7194 - val_auc: 0.7886
Epoch 4/100
9949/9949 [==============================] - 89s 9ms/step - loss: 0.5584 - binary_accuracy: 0.7208 - auc: 0.7854 - val_loss: 0.5541 - val_binary_accuracy: 0.7260 - val_auc: 0.7918
Epoch 5/100
9949/9949 [==============================] - 89s 9ms/step - loss: 0.5544 - binary_accuracy: 0.7239 - auc: 0.7894 - val_loss: 0.5505 - val_binary_accuracy: 0.7253 - val_auc: 0.7959
Epoch 6/100
9949/9949 [================

In [ ]:
# Evaluate model on test set
y_pred = model.predict(test_x)
roc_auc_score(test_y, y_pred)

3110/3110 [==============================] - 9s 3ms/step


0.7862643509662941

Best ROC AUC score (train) : 0.8703 </br>
Best ROC AUC score (validate) : 0.8093 </br>
Best ROC AUC score (test) : 0.7863 </br>